This code takes in the PM2.5 dataset collected from the US embassy in Beijing and transforms it from an hourly measurement to a daily measurement. Each scalar value is averaged for the entire day, except for cumulated rain and snow hours, of which the maximum is taken (if greater than 24 hours, take 24 hours instead)

In [1]:
#import libraries
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
uploaded = files.upload()

Saving PRSA_data_2010.1.1-2014.12.31.csv to PRSA_data_2010.1.1-2014.12.31.csv


In [3]:
df = pd.read_csv(open("PRSA_data_2010.1.1-2014.12.31.csv", "r"))

In [4]:
#FROM DATASET DESCRIPTION
#https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data
#No: row number
#year: year of data in this row
#month: month of data in this row
#day: day of data in this row
#hour: hour of data in this row
#pm2.5: PM2.5 concentration (ug/m^3)
#DEWP: Dew Point (â„ƒ)
#TEMP: Temperature (â„ƒ)
#PRES: Pressure (hPa)
#cbwd: Combined wind direction
#Iws: Cumulated wind speed (m/s)
#Is: Cumulated hours of snow
#Ir: Cumulated hours of rain
df.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [5]:
df.count()

No       43824
year     43824
month    43824
day      43824
hour     43824
pm2.5    41757
DEWP     43824
TEMP     43824
PRES     43824
cbwd     43824
Iws      43824
Is       43824
Ir       43824
dtype: int64

In [6]:
df['TEMP'].max()

42.0

In [7]:
df.dtypes

No         int64
year       int64
month      int64
day        int64
hour       int64
pm2.5    float64
DEWP       int64
TEMP     float64
PRES     float64
cbwd      object
Iws      float64
Is         int64
Ir         int64
dtype: object

In [8]:
df.tail()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0
43823,43824,2014,12,31,23,12.0,-21,-3.0,1034.0,NW,249.85,0,0


In [9]:
df[df['Ir']>5].head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
2113,2114,2010,3,30,1,288.0,6,7.0,1020.0,SE,2.68,0,6
2114,2115,2010,3,30,2,254.0,6,7.0,1020.0,SE,4.47,0,7
2115,2116,2010,3,30,3,254.0,6,7.0,1019.0,SE,6.26,0,8
2116,2117,2010,3,30,4,248.0,6,7.0,1018.0,SE,8.05,0,9
2117,2118,2010,3,30,5,271.0,6,7.0,1018.0,SE,9.84,0,10


In [10]:
#i know there's a better way to do this but i'm lazy as heck
def days_in_cumulative_months(x, year):
  if(year==2012):
    if(x==1):
      return 0
    elif(x==2):
      return 31
    elif(x==3):
      return 60
    elif(x==4):
      return 91
    elif(x==5):
      return 121
    elif(x==6):
      return 152
    elif(x==7):
      return 182
    elif(x==8):
      return 213
    elif(x==9):
      return 244
    elif(x==10):
      return 274
    elif(x==11):
      return 305
    else:
      return 335
  else:
    if(x==1):
      return 0
    elif(x==2):
      return 31
    elif(x==3):
      return 59
    elif(x==4):
      return 90
    elif(x==5):
      return 120
    elif(x==6):
      return 151
    elif(x==7):
      return 181
    elif(x==8):
      return 212
    elif(x==9):
      return 243
    elif(x==10):
      return 273
    elif(x==11):
      return 304
    else:
      return 334
  
def days_in_cumulative_years(year):
  if(year==2010):
    return 0
  elif(year==2011):
    return 365
  elif(year==2012):
    return 731
  elif(year==2013):
    return 1096
  else:
    return 1461

In [11]:
list_of_days = []
for index, row in df.iterrows():
  list_of_days.append(days_in_cumulative_years(row['year'])+days_in_cumulative_months(row['month'],row['year'])+row['day'])

In [12]:
df['day (cumulative)'] = list_of_days
df.tail()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,day (cumulative)
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0,1826
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0,1826
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0,1826
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0,1826
43823,43824,2014,12,31,23,12.0,-21,-3.0,1034.0,NW,249.85,0,0,1826


In [13]:
df2 = df.groupby('day (cumulative)').mean()[['pm2.5','DEWP','TEMP','PRES','Iws']]
df2.head()

,pm2.5,DEWP,TEMP,PRES,Iws
day (cumulative),,,,,
1,NaN,-18.750000,-6.750000,1017.083333,14.458333
2,145.958333,-8.500000,-5.125000,1024.750000,24.860000
3,78.833333,-10.125000,-8.541667,1022.791667,70.937917
4,31.333333,-20.875000,-11.500000,1029.291667,111.160833
5,42.458333,-24.583333,-14.458333,1033.625000,56.920000


In [14]:
cbwd_series = df.groupby('day (cumulative)')['cbwd'].agg(pd.Series.mode)
type(cbwd_series)

pandas.core.series.Series

In [15]:
df2['cbwd'] = cbwd_series
df2.head()

,pm2.5,DEWP,TEMP,PRES,Iws,cbwd
day (cumulative),,,,,,
1,NaN,-18.750000,-6.750000,1017.083333,14.458333,NW
2,145.958333,-8.500000,-5.125000,1024.750000,24.860000,SE
3,78.833333,-10.125000,-8.541667,1022.791667,70.937917,SE
4,31.333333,-20.875000,-11.500000,1029.291667,111.160833,NW
5,42.458333,-24.583333,-14.458333,1033.625000,56.920000,NW


In [16]:
#i think i just need to run this again
Is_series = df.groupby('day (cumulative)')['Is'].agg('max')
Ir_series = df.groupby('day (cumulative)')['Ir'].agg('max')
for i in Is_series:
  if(i>24):
    Is_series = Is_series.replace(i,24)
for i in Ir_series:
  if(i>24):
    Ir_series = Ir_series.replace(i,24)

Is_series.head()

day (cumulative)
1     0
2     4
3    24
4     0
5     0
Name: Is, dtype: int64

In [17]:
df2['Rain hours']=Ir_series
df2['Snow hours']=Is_series
df2.head()

,pm2.5,DEWP,TEMP,PRES,Iws,cbwd,Rain hours,Snow hours
day (cumulative),,,,,,,,
1,NaN,-18.750000,-6.750000,1017.083333,14.458333,NW,0,0
2,145.958333,-8.500000,-5.125000,1024.750000,24.860000,SE,0,4
3,78.833333,-10.125000,-8.541667,1022.791667,70.937917,SE,0,24
4,31.333333,-20.875000,-11.500000,1029.291667,111.160833,NW,0,0
5,42.458333,-24.583333,-14.458333,1033.625000,56.920000,NW,0,0


In [18]:
df3 = df2.copy()
df3 = df3.fillna(method='bfill')
df3.head()

,pm2.5,DEWP,TEMP,PRES,Iws,cbwd,Rain hours,Snow hours
day (cumulative),,,,,,,,
1,145.958333,-18.750000,-6.750000,1017.083333,14.458333,NW,0,0
2,145.958333,-8.500000,-5.125000,1024.750000,24.860000,SE,0,4
3,78.833333,-10.125000,-8.541667,1022.791667,70.937917,SE,0,24
4,31.333333,-20.875000,-11.500000,1029.291667,111.160833,NW,0,0
5,42.458333,-24.583333,-14.458333,1033.625000,56.920000,NW,0,0


In [19]:
df3.isna().sum()

pm2.5         0
DEWP          0
TEMP          0
PRES          0
Iws           0
cbwd          0
Rain hours    0
Snow hours    0
dtype: int64

In [20]:
compression_opts = dict(method='zip', archive_name='out.csv')  
df3.to_csv('out.zip', index=False, compression=compression_opts)

In [22]:
from google.colab import drive
drive.mount('drive')
df3.to_csv('PM25.csv')
!cp PM25.csv "drive/My Drive/"

Mounted at drive
